# Using AIfES Converter
Run this notebook after executing `TF-MNIST-Dense-Demo.ipynb` .

For more detail on AIfES see:
 - https://github.com/Fraunhofer-IMS/AIfES-Converter
 - https://fraunhofer-ims.github.io/AIfES-Converter/quickstart.html
 - https://github.com/Fraunhofer-IMS/AIfES_for_Arduino

Ensure Python 3.9 for AIfES

In [6]:
import sys
sys.version

'3.9.13 (tags/v3.9.13:6de2ca5, May 17 2022, 16:36:42) [MSC v.1929 64 bit (AMD64)]'

In [7]:
import numpy as np
from pathlib import Path
from tensorflow import keras
from aifes import keras2aifes

Network input data, required to set quanitzation range. Use Test data as representative_dataset.

In [8]:
# Load the data and split it between train and test sets.
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range.
x_test = x_test.astype("float32") / 255

# Flatten 28 * 28 images for Dense layer input.
representative_dataset = np.reshape(x_test, (10000, 784))

print("x_test shape: ", representative_dataset.shape)
print("test samples: ",representative_dataset.shape[0] )

x_test shape:  (10000, 784)
test samples:  10000


List of models

In [9]:
models_list = [
    "mnist-dense"
]

Load a model from the model list, use AIfESConverter and create a float and a q7 model. Parse the generated c files and exchange the generic variable, function and define names with model specific ones.

In [10]:
for model in models_list:
    keras_model = keras.models.load_model(model + '.h5')
    dest_path = Path('..\\aifes-gates-cube-project\\AIfES-models\\AIfES-')
    model_path = Path(str(dest_path) + model + '-float')
    model_name = model.replace('-', '_')
    keras2aifes.convert_to_fnn_f32_express(keras_model, str(model_path))
    
    # Adjust the name of the float model
    # AIfES always uses the same model name, this section changes the name of the model
    # and makes it possible to import multiple models within the same c code file.
    fnn_h = model_path / 'aifes_e_f32_fnn.h'
    with open(fnn_h,'r') as file:
        filedata = file.read()
        # Adjust function names
        filedata = filedata.replace('aifes_e_f32_fnn_inference(','aifes_e_' + model_name + '_f32_fnn_inference(')
        # Adjust variable names
        filedata = filedata.replace('(void*)aifes_e_f32_flat_weights;','(void*) model_' + model_name + '_flat_weights;')
        # Adjust defines
        filedata = filedata.replace('AIFES_E_F32_FNN','AIFES_E_F32_' + model_name.upper() + '_FNN')
    with open(fnn_h,'w') as file:
        file.write(filedata)
    
    weights_h = model_path / 'aifes_e_f32_weights.h'
    with open(weights_h,'r') as file:
        filedata = file.read()
        # Adjust variable names
        filedata = filedata.replace('aifes_e_f32_flat_weights[]','model_' + model_name + '_flat_weights[]')
        # Adjust defines
        filedata = filedata.replace('AIFES_E_F32_WEIGHTS','AIFES_E_F32_' + model_name.upper() + '_WEIGHTS')
    with open(weights_h,'w') as file:
        file.write(filedata)
    
    
    model_path = Path(str(dest_path) + model + '-q7')
    # Quantized Q7 model with 32 bit storage alignment (4 Byte) and little endian representation
    keras2aifes.convert_to_fnn_q7_express(keras_model, str(model_path), 
                                      representative_data=representative_dataset, 
                                      target_alignment=4, 
                                      byteorder="little")
    
    
    # Adjust the name of the q7 model
    # AIfES always uses the same model name, this section changes the name of the model
    # and makes it possible to import multiple models within the same c code file.
    fnn_h = model_path / 'aifes_e_q7_fnn.h'
    with open(fnn_h,'r') as file:
        filedata = file.read()
        # Adjust function names
        filedata = filedata.replace('aifes_e_q7_fnn_inference(','aifes_e_' + model_name + '_q7_fnn_inference(')
        # Adjust variable names
        filedata = filedata.replace('(void*) model_parameters;','(void*) model_' + model_name + '_parameters;')
        # Adjust defines
        filedata = filedata.replace('AIFES_E_Q7_FNN','AIFES_E_Q7_' + model_name.upper() + '_FNN')
    with open(fnn_h,'w') as file:
        file.write(filedata)
    
    weights_h = model_path / 'aifes_e_q7_weights.h'
    with open(weights_h,'r') as file:
        filedata = file.read()
        # Adjust variable names
        filedata = filedata.replace('model_parameters[','model_' + model_name + '_parameters[')
        filedata = filedata.replace('parameter_memory_size','parameter_' + model_name + '_memory_size')
        # Adjust defines
        filedata = filedata.replace('AIFES_E_Q7_WEIGHTS','AIFES_E_Q7_' + model_name.upper() + '_WEIGHTS')
    with open(weights_h,'w') as file:
        file.write(filedata)
    

## Generate Test Data in C
Test data to include in C program.

In [40]:
def numpy_to_c_array(ndarr, name="carray", d_type="auto"):
    if not isinstance(ndarr, np.ndarray):
        raise TypeError("Given instance is not of type numpy.ndarray")
    
    if d_type=="auto":
        if np.dtype(float) == ndarr.dtype:
            d_type = "float"
        elif np.dtype(np.int8) == ndarr.dtype:
            d_type = "int8_t"
        elif np.dtype(np.int16) == ndarr.dtype:
            d_type = "int16_t"
        elif np.dtype(np.int32) == ndarr.dtype:
            d_type = "int32_t"
        else:
            raise f"Unsupported data type of ndarr {ndarr.dtype}"
        
    return f"{d_type} {name}[] = \n" + numpy_to_c_array_data(ndarr) + "\n\n"
        
def numpy_to_c_array_data(arr, external_indent='  ', internal_indent='  ', columns=32):
    """Convert a numpy array to its C/C++ representation and returns the string."""
    indent = external_indent + internal_indent
    arr_str = indent + '{'
    for i in range(0,len(arr.flat) -1 ):
        arr_str = arr_str + f"{arr.flat[i]}, "
        if (i % columns == 0) and (i != 0):
            arr_str = arr_str + "\n" + indent
        
    arr_str = arr_str + f"{arr.flat[len(arr.flat) - 1]}}};"
    
    return arr_str

In [41]:
y_test = keras.utils.to_categorical(y_test, 10)

Create test data for each number (0-9).

In [42]:
c_data = numpy_to_c_array(representative_dataset[3], "mnist_number_0", "float");
c_data += numpy_to_c_array(representative_dataset[2], "mnist_number_1", "float");
c_data += numpy_to_c_array(representative_dataset[1], "mnist_number_2", "float");
c_data += numpy_to_c_array(representative_dataset[18], "mnist_number_3", "float");
c_data += numpy_to_c_array(representative_dataset[4], "mnist_number_4", "float");
c_data += numpy_to_c_array(representative_dataset[8], "mnist_number_5", "float");
c_data += numpy_to_c_array(representative_dataset[11], "mnist_number_6", "float");
c_data += numpy_to_c_array(representative_dataset[17], "mnist_number_7", "float");
c_data += numpy_to_c_array(representative_dataset[61], "mnist_number_8", "float");
c_data += numpy_to_c_array(representative_dataset[9], "mnist_number_9", "float");

Uncomment print to view c arrays.

In [45]:
#print(c_data)